In [ ]:
from matplotlib import pyplot as plt
import os
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
import cv2 
from sklearn.neural_network import MLPClassifier
from mlp_predict_tools import gerador_de_csv_com_as_coordenadas_e_os_rotulos
import joblib
# Usado para criar a Matrix de Confusão
from sklearn.metrics import confusion_matrix, accuracy_score
# %matplotlib inline # para jupyter notebook

In [ ]:
# Código para garantir que o nome de cada arquivo de imagem seja um número inteiro e
# que os números sigam uma sequencia monotonica iniciando em 1
# o diretório relativo a letra T possuía problemas com os nomes de algumas imagens
# NÂO é necessário executar o código dessa célula, a menos que o conjunto de dados 
# com os nomes originais seja usado
lista_de_arquivos = []
for nome_do_arquivo in os.listdir('imagens/train/T'):
         if os.path.isfile(os.path.join('imagens/train/T', nome_do_arquivo)):
             lista_de_arquivos.append(nome_do_arquivo)
lista_de_arquivos.sort()
indice_inicial = 413
for i in range(indice_inicial, len(lista_de_arquivos)):
    nome_atual = f'./imagens/train/T/{lista_de_arquivos[i]}'
    novo_indice = i + 1
    nome_novo = f'./imagens/train/T/{novo_indice}.png'
    os.rename(nome_atual, nome_novo)

In [ ]:
# Lista com nome de todos os diretórios
label = ["A","B","C","D","E","F","G","I","L","M","N","O","P","Q","R","S","T","U","V","W"]

# Landmarks: 
* WRIST
* THUMB_CMC
* THUMB_MCP
* THUMB_IP
* THUMB_TIP
* INDEX_FINGER_MCP
* INDEX_FINGER_PIP
* INDEX_FINGER_DIP
* INDEX_FINGER_TIP
* MIDDLE_FINGER_MCP
* MIDDLE_FINGER_PIP
* MIDDLE_FINGER_DIP
* MIDDLE_FINGER_TIP
* RING_FINGER_MCP
* RING_FINGER_PIP
* RING_FINGER_DIP
* RING_FINGER_TIP
* PINKY_MCP
* PINKY_PIP
* PINKY_DIP
* PINKY_TIP


In [ ]:
gerador_de_csv_com_as_coordenadas_e_os_rotulos('test', 'normalizado')
gerador_de_csv_com_as_coordenadas_e_os_rotulos('test', 'centro_geometrico')
gerador_de_csv_com_as_coordenadas_e_os_rotulos('train', 'normalizado')
gerador_de_csv_com_as_coordenadas_e_os_rotulos('train', 'centro_geometrico')

In [ ]:
df_train = pd.read_csv('train_centro_geometrico.csv')
X_train = df_train.drop(columns=['label'])
y_train = df_train['label']

In [ ]:
df_test = pd.read_csv('test_centro_geometrico.csv')
X_test = df_test.drop(columns=['label'])
y_test = df_test['label']

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100,100), random_state=1, max_iter = 10000)


In [ ]:
clf.fit(X_train, y_train)

In [ ]:
results = clf.score(X_test,y_test)

In [ ]:
resultado_test = clf.predict(X_test)

In [ ]:
class_test = y_test

In [ ]:
acuracia = accuracy_score(resultado_test, class_test)
print(f"\nacurácia: {acuracia*100}%")
conf_matrix = confusion_matrix(resultado_test, class_test)
print(f"\nMatrix de Confusão: {conf_matrix}")


# Visualização da Matrz de confusão com seaborn
fig, axes = plt.subplots(1, 2, figsize=(200, 100), sharey=True)
fig.suptitle("Matrizes de Confusão")

sns.heatmap(ax=axes[0], data=conf_matrix, annot=True)
axes[0].set_title("Matriz de Confusão")

sns.heatmap(ax=axes[1], data=conf_matrix/np.sum(conf_matrix), annot=True,
            fmt='.2%', cmap='Blues')
axes[1].set_title("Matriz de Confusão Percentual");
plt.show()


In [ ]:
print(results)

In [ ]:
# Salvar o modelo usando joblib
arquivo_joblib = "modelo_treinado_mediapipe.joblib"
joblib.dump(clf, arquivo_joblib)

In [ ]:
df_results_compare = pd.DataFrame(results == y_test)
df_results_compare['label'].value_counts()
    

In [1]:
import cv2
import numpy as np
import mediapipe as mp
from joblib import load
from interface_aplication_tools import display_classification, display_prediction
import time
import pandas as pd
from mlp_predict_tools import imagem_para_lista_de_coordenadas 

# Carregar o modelo treinado
model = load('modelo_treinado_mediapipe.joblib')

# Inicializar MediaPipe para detecção de mãos
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

imagem_para_lista_de_coordenadas()

# Inicializar a câmera
cap = cv2.VideoCapture(0)

# Último tempo de captura
last_capture_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Erro ao capturar o frame da câmera")
        break
    
    # Converter a imagem para RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Processar a imagem com MediaPipe
    result = hands.process(rgb_frame)

    # Mostrar a câmera com um label inicial
    display_prediction(frame)

    # Tempo atual
    current_time = time.time()
    
    # Verificar se 0.5 segundos passaram desde a última captura
    if current_time - last_capture_time >= 0.5:
        last_capture_time = current_time
        
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Pré-processar os landmarks
                landmarks_array = preprocess_landmarks(hand_landmarks)
                
                # Adicionar as coordenadas ao DataFrame
                df_coordinates = pd.concat([df_coordinates, pd.DataFrame(landmarks_array, columns=df_coordinates.columns)], ignore_index=True)
                
                # Fazer a previsão usando o modelo
                prediction = model.predict(landmarks_array)
                predicted_class = np.argmax(prediction)
                
                # Mapear a classe prevista para a letra correspondente
                classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W']
                predicted_letter = classes[predicted_class]

                # Exibir o resultado da classificação na câmera usando o OpenCV
                display_classification(frame, predicted_letter)

    # Aguardar um pequeno atraso antes de capturar o próximo frame
    time.sleep(0.02)

    # Fechar a câmera ao pressionar a tecla "q"
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        # Exportar o DataFrame para um arquivo CSV
        df_coordinates.to_csv('landmarks_coordinates.csv', index=False)
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\bruno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


ValueError: Shape of passed values is (1, 63), indices imply (1, 7)

: 